In [5]:
import re
import json
import time
import os
import sys
import numpy as np
import qiskit

from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.converters import circuit_to_dag

# Dynamically add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd())))  # Adjust path to root
sys.path.append(project_root)

from src.CE_module import construct_qcc_circuit, CE_recur_tree
from src.CA_module import extract_CNOT_network, update_probabilities, update_observables
from src.utilities import compare_lists, generate_pauli_strings
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers

In [3]:
#first compare the UCCSD ansatz
test_paulis = generate_UCCSD_entanglers(10, 20)
test_params = [0.01 * i for i in range(len(test_paulis))]

start_time = time.time()
opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = CE_recur_tree(entanglers=test_paulis, params=test_params, barrier=False)
opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
end_time = time.time()
combined_time = end_time - start_time

801


In [4]:
opt_qiskit.count_ops()

OrderedDict([('rz', 36679), ('cx', 24022), ('sx', 23637), ('x', 881)])

In [6]:
for num_obs in [10, 50, 100, 500, 1000, 5000]:
    random_paulis = generate_pauli_strings(20, num_obs)
    start_time = time.time()
    updated_signs, updated_observables = update_observables(random_paulis, [append_clifford_f2])
    end_time = time.time()
    compile_time = end_time - start_time
    print("num_obs", num_obs, "compile_time", compile_time)

num_obs 10 compile_time 0.05076193809509277
num_obs 50 compile_time 0.11133980751037598
num_obs 100 compile_time 0.2135317325592041
num_obs 500 compile_time 1.038912057876587
num_obs 1000 compile_time 2.0829989910125732
num_obs 5000 compile_time 10.279476165771484


In [7]:
#then compare the compile time for MAXCUT 
with open("../benchmarks/max_cut_paulis/max_cut_benchmark_regular_n20_e12_l1.json", "r") as file:
    test_paulis = json.load(file)
test_params = [0.01 * i for i in range(len(test_paulis))]

start_time = time.time()
opt_qc, append_clifford, sorted_entanglers = CE_recur_tree(entanglers=test_paulis, params=test_params, barrier=False)
opt_qiskit = transpile(opt_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
end_time = time.time()
opt_qiskit.measure_active()
combined_time = end_time - start_time

In [8]:
opt_qiskit.count_ops()

OrderedDict([('cx', 171),
             ('rz', 156),
             ('sx', 20),
             ('measure', 20),
             ('barrier', 1)])

In [9]:
simulator = AerSimulator()
cnot_network = extract_CNOT_network(append_clifford)
for num_states in [10, 50, 100, 500, 1000, 5000]:
    opt_result = simulator.run(opt_qiskit, shots = num_states).result()
    start_time = time.time()
    dag = circuit_to_dag(cnot_network)
    updated_probs = update_probabilities(opt_result.get_counts(), dag)
    end_time = time.time()
    compile_time = end_time - start_time
    print("num_states", num_states, "compile_time", compile_time)

num_states 10 compile_time 0.0034470558166503906
num_states 50 compile_time 0.008790016174316406
num_states 100 compile_time 0.016021013259887695
num_states 500 compile_time 0.07719922065734863
num_states 1000 compile_time 0.14345502853393555
num_states 5000 compile_time 0.562302827835083
